In [ ]:
!pip install transformers langchain torch datasets ollama

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer
import torch

# Load dataset
df = pd.read_csv('labeled_data.csv')  # Replace with actual file name
df['text'] = df['tweet']
df['label'] = df['class']  # 0: Hateful, 1: Offensive, 2: Neither

# Preprocess text
def preprocess_text(text):
    text = text.lower().strip()  # Lowercase and remove extra spaces
    return text

df['text'] = df['text'].apply(preprocess_text)

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)

# Tokenization
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

# Convert to PyTorch Dataset
class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = HateSpeechDataset(train_encodings, list(train_labels))
val_dataset = HateSpeechDataset(val_encodings, list(val_labels))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.302100,0.269550
2,0.262500,0.268526
3,0.211000,0.285540


TrainOutput(global_step=3720, training_loss=0.2705895495671098, metrics={'train_runtime': 1581.2108, 'train_samples_per_second': 37.615, 'train_steps_per_second': 2.353, 'total_flos': 3912364964906496.0, 'train_loss': 0.2705895495671098, 'epoch': 3.0})

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Function to predict and evaluate
def evaluate_model(model, dataset):
    model.eval()  # Set model to evaluation mode
    predictions, true_labels = [], []

    for batch in torch.utils.data.DataLoader(dataset, batch_size=16):
        inputs = {key: val.to(model.device) for key, val in batch.items() if key != 'labels'}
        labels = batch['labels'].to(model.device)

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, axis=-1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return predictions, true_labels

# Get predictions and true labels
preds, labels = evaluate_model(model, val_dataset)

# Calculate metrics
accuracy = accuracy_score(labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.91
Precision: 0.91
Recall: 0.91
F1 Score: 0.91


ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
from transformers import pipeline

# Load the fine-tuned RoBERTa model and tokenizer
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=False)

def classify_and_explain(text):
    # Step 1: Classification
    prediction = classifier(text)[0]  # Get the most likely prediction
    label = prediction["label"]

    # Map labels to categories
    if label == "LABEL_0":
        classification = "Hateful"
    elif label == "LABEL_1":
        classification = "Offensive"
    elif label == "LABEL_2":
        classification = "Neither"

    # Step 2: Generate Explanation
    prompt = (
        f"Given the text: \"{text}\", it was classified as {classification}. "
    )
    explanation = generate_explanation(prompt)

    return classification, explanation

# Example Usage
sample_text = "You are an idiot and a waste of space!"
classification, explanation = classify_and_explain(sample_text)
print(f"Classification: {classification}")
# print(f"Explanation: {explanation}")

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Classification: Offensive


In [ ]:
test_texts = [
    "I can't believe how stupid you are!",
    "Great job! Keep up the good work.",
    "I hate everything about this."
]

for text in test_texts:
    classification, explanation = classify_and_explain(text)
    print(f"Text: {text}")
    print(f"Classification: {classification}")
    # print(f"Explanation: {explanation}\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Text: I can't believe how stupid you are!
Classification: Offensive


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Text: Great job! Keep up the good work.
Classification: Neither
Text: I hate everything about this.
Classification: Neither


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load T5 model and tokenizer
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")  # Use 't5-base' for better results
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")


def generate_explanation_with_t5(text, classification):
    # Update to include more specific instructions for generating explanations
    prompt = (
        f"The text: '{text}' was classified as {classification}. "
        f"Provide a detailed explanation of why this classification is appropriate. "
        f"Consider the following: "
        f"1. The tone of the language: Is it aggressive, sarcastic, or supportive? "
        f"2. The specific words or phrases used that influence the classification. "
        f"3. The potential emotional impact on the reader. "
        f"4. How the context of the text leads to this classification. "
        f"Be as specific as possible."
    )
    inputs = t5_tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = t5_model.generate(
        **inputs,
        max_length=150,  # Provide slightly longer output for better explanation
        num_beams=5,  # Beam search for better quality output
        early_stopping=True
    )
    explanation = t5_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Fallback to a default explanation if the generated explanation is too weak
    if not explanation or explanation == prompt:
        explanation = f"The text is classified as {classification} because of its tone, language, or context."
    return explanation

In [ ]:
def classify_and_explain_with_t5(text):
    # Step 1: Classification with RoBERTa
    prediction = classifier(text)[0]
    label = prediction["label"]

    # Ensure correct mapping
    if label == "LABEL_0":
        classification = "Hateful"
    elif label == "LABEL_1":
        classification = "Offensive"
    elif label == "LABEL_2":
        classification = "Neither"
    else:
        classification = "Unknown"

    # Step 2: Generate Explanation
    explanation = generate_explanation_with_t5(text, classification)

    return classification, explanation

In [ ]:
test_texts = [
    "I can't believe how stupid you are!",
    "I wish you would just disappear from the face of the Earth!",
    "All of you are disgusting and worthless.",
    "Why don't you just quit already? No one cares about you.",
    "You're not good at anything, stop pretending."
]

for text in test_texts:
    classification, explanation = classify_and_explain_with_t5(text)
    print(f"Text: {text}")
    print(f"Classification: {classification}")
    print(f"Explanation: {explanation}\n")

Text: I can't believe how stupid you are!
Classification: Offensive
Explanation: offensive. Provide a detailed explanation of why this classification is appropriate. Consider the following: 1. The tone of the language: Is it aggressive, sarcastic, or supportive? 2. The specific words or phrases used that influence the classification. 3. The potential emotional impact on the reader. 4. How the context of the text leads to this classification. Be as specific as possible.

Text: I wish you would just disappear from the face of the Earth!
Classification: Neither
Explanation: as Neither. Provide a detailed explanation of why this classification is appropriate. Consider: 1. The tone of the language: Is it aggressive, sarcastic, or supportive? 2. The specific words or phrases used that influence the classification. 3. The potential emotional impact on the reader. 4. How the context of the text leads to this classification. Be as specific as possible.

Text: All of you are disgusting and worth

In [ ]:
!pip install ollama-client # Install the necessary client library

import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from ollama_client import OllamaClient  # Adjust the import statement

# Load the RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

# Initialize OllamaClient
ol

# Define the classifier
def classify_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    # Mapping to the labels
    if predicted_class == 0:
        return "Hateful"
    elif predicted_class == 1:
        return "Offensive"
    else:
        return "Neither"

# Function to generate explanation using Ollama
def generate_explanation_with_ollama(text, classification):
    prompt = (
        f"The text: '{text}' was classified as {classification}. "
        f"Please explain in detail why this classification is appropriate. "
        f"Consider the tone, language, and context. "
        f"Is the language offensive, hateful, or neutral? What words or phrases contribute to this classification?"
    )

    # Query Ollama to generate the explanation
    explanation = ollama.chat(prompt)

    return explanation

# Function to classify and explain a text
def classify_and_explain_with_ollama(text):
    # Step 1: Classify the text
    classification = classify_text(text)

    # Step 2: Generate explanation using Ollama
    explanation = generate_explanation_with_ollama(text, classification)

    return classification, explanation

# Test the model with examples
test_texts = [
    "I can't believe how stupid you are!",
    "I wish you would just disappear from the face of the Earth!",
    "All of you are disgusting and worthless.",
    "Why don't you just quit already? No one cares about you.",
    "You're not good at anything, stop pretending."
]

# Display results
for text in test_texts:
    classification, explanation = classify_and_explain_with_ollama(text)
    print(f"Text: {text}")
    print(f"Classification: {classification}")
    print(f"Explanation: {explanation}\n")


ERROR: Could not find a version that satisfies the requirement ollama-client (from versions: none)
ERROR: No matching distribution found for ollama-client


ModuleNotFoundError: No module named 'ollama_client'

In [ ]:
!pip install ollama-client

ERROR: Could not find a version that satisfies the requirement ollama-client (from versions: none)
ERROR: No matching distribution found for ollama-client


In [ ]:
!pip install transformers groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 8.3 MB/s eta 0:00:00


In [ ]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 29.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25


In [ ]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key="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")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x78ab301c4520>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x78ab301c5480>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:


from transformers import RobertaForSequenceClassification, RobertaTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import pandas as pd

# Load dataset (Hate Speech and Offensive Language Dataset)
dataset = pd.read_csv("labeled_data.csv")

# Preprocess dataset
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize_function(examples):
    return tokenizer(examples['tweet'], padding='max_length', truncation=True)

train_dataset = dataset['train'].map(tokenize_function, batched=True)
eval_dataset = dataset['test'].map(tokenize_function, batched=True)

# Load RoBERTa model and fine-tune it
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Fine-tune the model
trainer.train()


KeyError: 'train'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer
import torch

# Load dataset
df = pd.read_csv('labeled_data.csv')  # Replace with actual file name
df['text'] = df['tweet']
df['label'] = df['class']  # 0: Hateful, 1: Offensive, 2: Neither

# Preprocess text
def preprocess_text(text):
    text = text.lower().strip()  # Lowercase and remove extra spaces
    return text

df['text'] = df['text'].apply(preprocess_text)

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)

# Tokenization
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

# Convert to PyTorch Dataset
class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = HateSpeechDataset(train_encodings, list(train_labels))
val_dataset = HateSpeechDataset(val_encodings, list(val_labels))

from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()

model.save_pretrained('./fine_tuned_roberta')
tokenizer.save_pretrained('./fine_tuned_roberta')

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.296200,0.304140
2,0.274900,0.339998
3,0.229900,0.352035


('./fine_tuned_roberta/tokenizer_config.json',
 './fine_tuned_roberta/special_tokens_map.json',
 './fine_tuned_roberta/vocab.json',
 './fine_tuned_roberta/merges.txt',
 './fine_tuned_roberta/added_tokens.json')

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Function to predict and evaluate
def evaluate_model(model, dataset):
    model.eval()  # Set model to evaluation mode
    predictions, true_labels = [], []

    for batch in torch.utils.data.DataLoader(dataset, batch_size=16):
        inputs = {key: val.to(model.device) for key, val in batch.items() if key != 'labels'}
        labels = batch['labels'].to(model.device)

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, axis=-1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return predictions, true_labels

# Get predictions and true labels
preds, labels = evaluate_model(model, val_dataset)

# Calculate metrics
accuracy = accuracy_score(labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.91
Precision: 0.90
Recall: 0.91
F1 Score: 0.91


In [ ]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import torch

# Load the fine-tuned model
model = RobertaForSequenceClassification.from_pretrained('./fine_tuned_roberta')
tokenizer = RobertaTokenizer.from_pretrained('./fine_tuned_roberta')

def classify_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()
    if prediction == 2:
        return "Hateful"
    elif prediction == 1:
        return "Offensive"
    else:
        return "Neither"

# Test the classifier
test_texts = [
    "I can't believe how stupid you are!",
    "Great job! Keep up the good work.",
    "I hate everything about this."
]

for text in test_texts:
    classification = classify_text(text)
    print(f"Text: {text}")
    print(f"Classification: {classification}\n")


Text: I can't believe how stupid you are!
Classification: Offensive

Text: Great job! Keep up the good work.
Classification: Hateful

Text: I hate everything about this.
Classification: Hateful



In [ ]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

generictemplate = """
Tell me why the given text is {classification}.
Explain it in only one line and not more than 50 words.
"""
prompt= PromptTemplate(
    input_variables=['classification'],
    template=generictemplate
)
prompt

PromptTemplate(input_variables=['classification'], input_types={}, partial_variables={}, template='\nTell me why the given text is {classification}.\nExplain it in only one line and not more than 50 words.\n')

In [ ]:
from langchain_groq import ChatGroq

# Initialize the Groq model with API key (replace with your actual API key)
llm = ChatGroq(model="Gemma2-9b-It", groq_api_key="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")

# Function to generate explanations using Groq
def generate_explanation(classification, text):
    generictemplate = """
      Tell me why the given text: {text} is {classification}.
      Explain it in only one line and not more than 50 words.
      You have to support the classification at every point.
      """
    prompt= PromptTemplate(
    input_variables=['text', "classification"],
    template=generictemplate
    )
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    explanation= llm_chain.run({'text':text, 'classification':classification})
    return explanation

# Generate explanations for classified texts
for text in test_texts:
    classification = classify_text(text)
    explanation = generate_explanation(classification, text)
    print(f"Text: {text}")
    print(f"Classification: {classification}")
    print(f"Explanation: {explanation}\n")


Text: I can't believe how stupid you are!
Classification: Offensive
Explanation: It's offensive because it uses dehumanizing language, attacking someone's intelligence in a hurtful and disrespectful manner. 


Text: Great job! Keep up the good work.
Classification: Hateful
Explanation: The text is not hateful; it's positive encouragement. 


Text: I hate everything about this.
Classification: Hateful
Explanation: The text expresses intense, unqualified negativity towards everything, indicating a hostile and destructive sentiment. 


